In [1]:
import geopandas as gpd
import pandas as pd
import fiona

#display
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import folium
plt.rcParams["figure.figsize"] = (15,15)
from IPython.display import clear_output
import contextily as cx

import sys
sys.path.append("../")
import src.supportExtract as se

import sys
sys.path.append("../")
import src.supportClean as sc

import sys
sys.path.append("../")
import src.bibliotheque as bb

import sys
sys.path.append("../")
import src.supportCharts as sch

# data input
let's see what layers are on the geopackage:

In [2]:
path_gpckg = '../data/catastro/catastro_madrid.gpkg'
dict_catastro = se.catastroLayers(path_gpckg)

the geopackage has layers: ['CadastralParcel', 'CadastralZoning', 'BuildingPart', 'Building', 'OtherConstruction', 'Address']
adding layer CadastralParcel to the output dictionary...
done
adding layer CadastralZoning to the output dictionary...
done
adding layer BuildingPart to the output dictionary...
done
adding layer Building to the output dictionary...
done
adding layer OtherConstruction to the output dictionary...
done
adding layer Address to the output dictionary...
done


In [5]:
gdf_CadastralParcel= dict_catastro[0]
gdf_CadastralZoning= dict_catastro[1]
gdf_BuildingPart = dict_catastro[2]
gdf_Building = dict_catastro[3]
gdf_OtherConstruction = dict_catastro[4]
gdf_Address = dict_catastro[5]

the variables that come form the Spanish Cadaster are:
- `cad_tot_houses` -> total number of houses per parcel
<br>the values go from 0 to 1750, being the median value around 250. that there are nulls in the `cad_tot_houses` column could mean that the building is not residential.
- `cad_tot_hab` -> total inhabitants
<br>the inhabitants per building? not sure if this data is cadastral. It can be the inhabitants of the area the buiding is located.
- `cad_tot_hotels` -> total number of hotels (buildigs that are classified as hotels by the cadaster)
<br>number of hotels in the area. The avg is zero, depending on the area of the city you're evaluating.
- `cad_med_consq` -> median value of the construction quality of the cadastral horizontal division
<br>is a 1-10 scale, so it is the median rating that the buildings have in the area, based on the construction techniques. The median value is a bit bigger than 4.
- `cad_tot_parcs` -> land surface
<br>is the plot that the buildings are build upon. The normal value is less than 50sqm? 
- `cad_pk` ->
- `cad_tot_srf_houses` -> houses' surface
<br>the built surface of the house classification. The median value is less than 25000 sqm.
- `cad_tot_srf_hotels` -> hotels' surface. 
<br> the built surface of hotels. The median value is around 0, as we've seen the hotel count is small.

In [6]:
print(gdf_CadastralParcel.shape)
gdf_CadastralParcel.head(5)

(141544, 11)


,gml_id,areaValue,areaValue_uom,beginLifespanVersion,endLifespanVersion,localId,namespace,label,nationalCadastralReference,pos,geometry
0,ES.SDGC.CP.000200500VK56E,1268,m2,2013-10-17T00:00:00,NaN,000200500VK56E,ES.SDGC.CP,000200500VK56E,000200500VK56E,451607.03 4467199.27,"MULTIPOLYGON (((961217.399 4481193.520, 961202..."
1,ES.SDGC.CP.000205600VK56E,297,m2,2013-10-17T00:00:00,NaN,000205600VK56E,ES.SDGC.CP,000205600VK56E,000205600VK56E,451592.44 4467170.83,"MULTIPOLYGON (((961211.833 4481181.394, 961197..."
2,ES.SDGC.CP.000205700VK56E,155,m2,2013-10-17T00:00:00,NaN,000205700VK56E,ES.SDGC.CP,000205700VK56E,000205700VK56E,451587.93 4467161.77,"MULTIPOLYGON (((961209.303 4481174.850, 961195..."
3,ES.SDGC.CP.000205800VK56E,174,m2,2013-10-17T00:00:00,NaN,000205800VK56E,ES.SDGC.CP,000205800VK56E,000205800VK56E,451584.62 4467155.04,"MULTIPOLYGON (((961206.406 4481167.358, 961192..."
4,ES.SDGC.CP.000205900VK56E,172,m2,2013-10-17T00:00:00,NaN,000205900VK56E,ES.SDGC.CP,000205900VK56E,000205900VK56E,451581.09 4467147.85,"MULTIPOLYGON (((961203.508 4481159.885, 961189..."


In [7]:
print(gdf_CadastralZoning.shape)
gdf_CadastralZoning.head(5)

(12347, 13)


,gml_id,beginLifespanVersion,endLifespanVersion,estimatedAccuracy,estimatedAccuracy_uom,localId,namespace,label,LocalisedCharacterString,nationalCadastalZoningReference,originalMapScaleDenominator,pos,geometry
0,ES.SDGC.CP.Z.28900A000,2019-04-12T00:00:00,NaN,1.0,m,28900A000,ES.SDGC.CP.Z,000,POLIGONO,28900A000,5000,440302.42 4492704.6,"MULTIPOLYGON (((948177.040 4505957.695, 948177..."
1,ES.SDGC.CP.Z.28900A001,2019-04-11T00:00:00,NaN,1.0,m,28900A001,ES.SDGC.CP.Z,001,POLIGONO,28900A001,5000,444909.27 4497953.5,"MULTIPOLYGON (((951975.321 4512908.139, 951989..."
2,ES.SDGC.CP.Z.28900A002,2013-10-17T00:00:00,NaN,1.0,m,28900A002,ES.SDGC.CP.Z,002,POLIGONO,28900A002,5000,446474.92 4496527.94,"MULTIPOLYGON (((953565.188 4510134.958, 953559..."
3,ES.SDGC.CP.Z.28900A003,2014-10-17T00:00:00,NaN,1.0,m,28900A003,ES.SDGC.CP.Z,003,POLIGONO,28900A003,5000,447749.11 4494280.98,"MULTIPOLYGON (((956903.888 4507489.306, 956903..."
4,ES.SDGC.CP.Z.28900A004,2013-10-17T00:00:00,NaN,1.0,m,28900A004,ES.SDGC.CP.Z,004,POLIGONO,28900A004,5000,447065.76 4493887.71,"MULTIPOLYGON (((953513.120 4508677.648, 953504..."


In [8]:
print(gdf_BuildingPart.shape)
gdf_BuildingPart.head(5)

(851937, 14)


,gml_id,beginLifespanVersion,conditionOfConstruction,localId,namespace,horizontalGeometryEstimatedAccuracy,horizontalGeometryEstimatedAccuracy_uom,horizontalGeometryReference,referenceGeometry,numberOfFloorsAboveGround,heightBelowGround,heightBelowGround_uom,numberOfFloorsBelowGround,geometry
0,ES.SDGC.BU. VK4700H_part1,2021-09-24T00:00:00,NaN,VK4700H_part1,ES.SDGC.BU,0.1,m,footPrint,True,3,0,m,0,"POLYGON ((949290.602 4484031.865, 949297.070 4..."
1,ES.SDGC.BU. VK4700H_part2,2021-09-24T00:00:00,NaN,VK4700H_part2,ES.SDGC.BU,0.1,m,footPrint,True,3,0,m,0,"POLYGON ((949213.578 4484057.098, 949213.504 4..."
2,ES.SDGC.BU. VK4700H_part3,2021-11-10T00:00:00,NaN,VK4700H_part3,ES.SDGC.BU,0.1,m,footPrint,True,5,0,m,0,"POLYGON ((949042.902 4483980.822, 949045.075 4..."
3,ES.SDGC.BU. VK4700H_part4,2021-11-10T00:00:00,NaN,VK4700H_part4,ES.SDGC.BU,0.1,m,footPrint,True,4,0,m,0,"POLYGON ((949042.902 4483980.822, 949043.532 4..."
4,ES.SDGC.BU. VK4700H_part5,2022-04-21T00:00:00,NaN,VK4700H_part5,ES.SDGC.BU,0.1,m,footPrint,True,4,0,m,0,"POLYGON ((949765.011 4484144.828, 949765.227 4..."


In [9]:
print(gdf_Building.shape)
gdf_Building.head(5)

(124578, 25)


,gml_id,beginLifespanVersion,conditionOfConstruction,beginning,end,endLifespanVersion,informationSystem,reference,localId,namespace,horizontalGeometryEstimatedAccuracy,horizontalGeometryEstimatedAccuracy_uom,horizontalGeometryReference,referenceGeometry,currentUse,numberOfBuildingUnits,numberOfDwellings,numberOfFloorsAboveGround,documentLink,format,sourceStatus,officialAreaReference,value,value_uom,geometry
0,ES.SDGC.BU. VK4700H,2022-05-10T00:00:00,-,--01-01T00:00:00,--01-01T00:00:00,NaN,https://www1.sedecatastro.gob.es/CYCBienInmueb...,VK4700H,VK4700H,ES.SDGC.BU,0.1,m,footPrint,True,NaN,0,0,NaN,http://ovc.catastro.meh.es/OVCServWeb/OVCWcfLi...,jpeg,NotOfficial,grossFloorArea,0,m2,"MULTIPOLYGON (((948922.858 4484281.431, 948922..."
1,ES.SDGC.BU.RA610010000000,2016-11-16T00:00:00,functional,1959-01-01T00:00:00,1991-01-01T00:00:00,NaN,https://www1.sedecatastro.gob.es/CYCBienInmueb...,RA610010000000,RA610010000000,ES.SDGC.BU,0.1,m,footPrint,True,1_residential,1,1,NaN,http://ovc.catastro.meh.es/OVCServWeb/OVCWcfLi...,jpeg,NotOfficial,grossFloorArea,105,m2,"POLYGON ((955006.443 4494903.221, 955005.247 4..."
2,ES.SDGC.BU.00006Z8VK4800A,2017-05-11T00:00:00,-,--01-01T00:00:00,--01-01T00:00:00,NaN,https://www1.sedecatastro.gob.es/CYCBienInmueb...,00006Z8VK4800A,00006Z8VK4800A,ES.SDGC.BU,0.1,m,footPrint,True,NaN,0,0,NaN,http://ovc.catastro.meh.es/OVCServWeb/OVCWcfLi...,jpeg,NotOfficial,grossFloorArea,0,m2,"POLYGON ((948772.642 4493092.204, 948772.046 4..."
3,ES.SDGC.BU.000200100VK48E,2004-05-19T00:00:00,functional,1968-01-01T00:00:00,1968-01-01T00:00:00,NaN,https://www1.sedecatastro.gob.es/CYCBienInmueb...,000200100VK48E,000200100VK48E,ES.SDGC.BU,0.1,m,footPrint,True,2_agriculture,1,0,NaN,http://ovc.catastro.meh.es/OVCServWeb/OVCWcfLi...,jpeg,NotOfficial,grossFloorArea,26,m2,"POLYGON ((949919.012 4500395.315, 949919.982 4..."
4,ES.SDGC.BU.000200500VK56E,2013-10-17T00:00:00,declined,1995-01-01T00:00:00,1995-01-01T00:00:00,NaN,https://www1.sedecatastro.gob.es/CYCBienInmueb...,000200500VK56E,000200500VK56E,ES.SDGC.BU,0.1,m,footPrint,True,3_industrial,1,0,NaN,http://ovc.catastro.meh.es/OVCServWeb/OVCWcfLi...,jpeg,NotOfficial,grossFloorArea,646,m2,"MULTIPOLYGON (((961202.515 4481199.009, 961203..."


In [10]:
gdf_Building['currentUse'].value_counts()

currentUse
1_residential         110113
4_3_publicServices      3820
3_industrial            3544
4_2_retail              2787
4_1_office              2218
2_agriculture            304
Name: count, dtype: int64

`df_Building` is the layer that has the info on the buidings type. However, there's information missing on hotels so the layer needs to be enriched.

In [11]:
print(gdf_OtherConstruction.shape)
gdf_OtherConstruction.head(5)

(14517, 7)


,gml_id,beginLifespanVersion,conditionOfConstruction,localId,namespace,constructionNature,geometry
0,ES.SDGC.BU.0006601VK3800E_PI.1,2017-07-07T00:00:00,NaN,0006601VK3800E_PI.1,ES.SDGC.BU,openAirPool,"POLYGON ((938674.985 4493003.093, 938672.905 4..."
1,ES.SDGC.BU.0007201VK3800E_PI.2,2003-01-02T00:00:00,NaN,0007201VK3800E_PI.2,ES.SDGC.BU,openAirPool,"POLYGON ((938584.906 4493137.993, 938589.317 4..."
2,ES.SDGC.BU.0007202VK3800E_PI.3,2006-06-19T00:00:00,NaN,0007202VK3800E_PI.3,ES.SDGC.BU,openAirPool,"POLYGON ((938671.295 4493110.708, 938671.770 4..."
3,ES.SDGC.BU.0007203VK3800E_PI.4,2003-05-20T00:00:00,NaN,0007203VK3800E_PI.4,ES.SDGC.BU,openAirPool,"POLYGON ((938606.606 4493042.625, 938604.116 4..."
4,ES.SDGC.BU.0007205VK3800E_PI.5,2012-03-14T00:00:00,NaN,0007205VK3800E_PI.5,ES.SDGC.BU,openAirPool,"POLYGON ((938555.746 4493057.894, 938554.869 4..."


In [12]:
print(gdf_Address.shape)
gdf_Address.head(5)

(99999, 12)


,gml_id,localId,namespace,specification,method,default,designator,type,level,validFrom,beginLifespanVersion,geometry
0,ES.SDGC.AD.28.900.10.1.1730101VK4813B,28.900.10.1.1730101VK4813B,ES.SDGC.AD,Entrance,fromFeature,True,1,1,siteLevel,NaN,2003-01-14T00:00:00,POINT (950119.405 4496166.440)
1,ES.SDGC.AD.28.900.10.10.1630113VK4813B,28.900.10.10.1630113VK4813B,ES.SDGC.AD,Entrance,fromFeature,True,10,1,siteLevel,NaN,2001-12-13T00:00:00,POINT (950115.452 4496138.670)
2,ES.SDGC.AD.28.900.10.12.1630114VK4813B,28.900.10.12.1630114VK4813B,ES.SDGC.AD,Entrance,fromFeature,True,12,1,siteLevel,NaN,2006-01-04T00:00:00,POINT (950113.695 4496124.983)
3,ES.SDGC.AD.28.900.10.3.1730101VK4813B,28.900.10.3.1730101VK4813B,ES.SDGC.AD,Parcel,fromFeature,True,3,1,siteLevel,NaN,2003-01-14T00:00:00,POINT (950127.006 4496167.550)
4,ES.SDGC.AD.28.900.10.6.1630111VK4813B,28.900.10.6.1630111VK4813B,ES.SDGC.AD,Entrance,fromFeature,True,6,1,siteLevel,NaN,2001-12-13T00:00:00,POINT (950115.927 4496157.191)


# data input hotels

In [13]:
gdf_hoteles = gpd.read_file('../output/geojson/hotels.geojson')
gdf_hoteles.head()

,@fechaActualizacion,@id,language,name,email,phone,fax,title,body,web,address,zipcode,locality,country,latitude,longitude,subAdministrativeArea,fechas,cod1_num,cod1_txt,cod2_num,cod2_txt,cod3_num,cod3_txt,cod2_txt_en,cod1_txt_en,cod3_txt_en,geometry
0,2023-04-12,101557,es,JW Marriott Madrid,NaN,+34 914 18 97 70,NaN,JW Marriott Madrid,<p><strong>Ubicado junto a la Plaza de Canalej...,https://www.esmadrid.com/alojamientos/jw-marri...,"Sevilla, 2",28014,Madrid,Spain,40.416802,-3.700126,Madrid,NaN,3,Alojamientos,7022.0,Hoteles,7027.0,5 estrellas,Hotels,Accommodation,5.0,POINT (-3.70013 40.41680)
1,2023-01-31,100999,es,UMusic Hotel Madrid,Socialmedia@umusic-hotels.com,+34 911 231 099,NaN,UMusic Hotel Madrid,"<p><strong>En pleno centro de Madrid, en la ca...",https://www.esmadrid.com/alojamientos/umusic-h...,"La Paz, 11,",28012,Madrid,Spain,40.415345,-3.704102,Madrid,NaN,3,Alojamientos,7022.0,Hoteles,7027.0,5 estrellas,Hotels,Accommodation,5.0,POINT (-3.70410 40.41534)
2,2022-11-07,100952,es,Radisson RED Madrid,info.madrid@radissonred.com,+34 912 98 48 00,NaN,Radisson RED Madrid,"<p><strong>La calle Atocha N&ordm;123, junto a...",https://www.esmadrid.com/alojamientos/radisson...,"de Atocha, 123",28012,Madrid,Spain,40.410038,-3.693744,Madrid,NaN,3,Alojamientos,7022.0,Hoteles,7026.0,4 estrellas,Hotels,Accommodation,4.0,POINT (-3.69374 40.41004)
3,2022-09-16,100602,es,Akeah,reservas@akeah.com,+34 919 994 177,NaN,Akeah,<p><strong>Situado en la calle de San Bernardo...,https://www.esmadrid.com/alojamientos/akeah,"San Bernardo, 48",28015,Madrid,Spain,40.424884,-3.707327,Madrid,NaN,3,Alojamientos,7022.0,Hoteles,7026.0,4 estrellas,Hotels,Accommodation,4.0,POINT (-3.70733 40.42488)
4,2022-09-26,100590,es,Thompson Madrid,thompsonmadrid@thompsonhotels.com,+34 910 62 12 34,NaN,Thompson Madrid,"<p><strong>Thompson Madrid, perteneciente a la...",https://www.esmadrid.com/alojamientos/thompson...,"del Carmen, s/n.",28013,Madrid,Spain,40.418580,-3.703415,Madrid,NaN,3,Alojamientos,7022.0,Hoteles,7028.0,5 estrellas Gran Lujo,Hotels,Accommodation,6.0,POINT (-3.70342 40.41858)


# spatial join
I will then join patially the coordinates from the hotels from ayto madrid and the Buildings extracted from catastro. Let's see the what crs is the input.

In [14]:
gdf_Building.crs

<Projected CRS: EPSG:25829>
Name: ETRS89 / UTM zone 29N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 12°W and 6°W: Faroe Islands - onshore and offshore; Ireland - offshore; Jan Mayen - onshore and offshore; Portugal - onshore and offshore; Spain - onshore and offshore; United Kingdom - UKCS offshore.
- bounds: (-12.0, 34.91, -6.0, 74.13)
Coordinate Operation:
- name: UTM zone 29N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [16]:
gdf_Building.to_crs(epsg=4326, inplace = True)

In [21]:
gdf_Building_hoteles = gpd.sjoin(gdf_Building, gdf_hoteles, how='left')
#['@id'].unique()

array([    nan,  67453.,  89532.,  67470.,  67500.,  67322.,  67546.,
        67790.,  67557.,  67487.,  67446.,  67861.,  94085.,  67468.,
        91932., 100602.,  67842.,  67623.,  67451.,  67560.,  67307.,
        67588.,  67297.,  67668.,  67422.,  67849.,  67867.,  67559.,
        67540.,  67425.,  86753.,  67567.,  67392.,  67839., 100999.,
        67330.,  67582.,  67742.,  67346.,  67264.,  67517.,  67450.,
        97116.,  67543.,  67529.,  82647.,  67786.,  67824.,  67741.,
        67775.,  67385.,  67793.,  67581.,  90190.,  67577.,  67561.,
        67615.,  67334.,  67262.,  67594.,  67535.,  67856.,  67405.,
        67583.,  67325.,  67497.,  85114.,  67282.,  67736.,  67712.,
        67331.,  67257.,  95433.,  67568.,  67427.,  67531.,  87832.,
        67751.,  67499.,  67267.,  84730.,  67404.,  67754.,  67336.,
        67658.,  67735.,  67743.,  67324.,  67731.,  67196.,  94625.,
        94624.,  67246.,  67245.,  67536.,  67489.,  67562.,  67384.,
        67271.,  675

In [ ]:
gdf_Building['@id'].value_counts()